In [7]:
# Add at the beginning of your notebook to make imports work
import sys
import os

# Get the absolute path to the project root directory (one level up from notebooks)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added {project_root} to Python path")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from concurrent.futures import ProcessPoolExecutor
from itertools import product


Added /Users/amber/Desktop/Simulation/Avail-Fusion-CryptoEcon to Python path


In [8]:
# Import necessary modules from the Avail-Fusion-CryptoEcon project
from model.stochastic_processes import create_price_with_volatility_strike
from model.cold_start import policy_cold_start_staking  # For reference to check flow calculations
from config.config import TIMESTEPS, DELTA_TIME
from config.initialize_simulation import initialize_state
import experiments.simulation_configuration as simulation  # For simulation parameters

# Set plot style
plt.style.use('ggplot')
sns.set_palette("viridis")

ModuleNotFoundError: No module named 'experiments'

In [ ]:
def generate_price_scenarios():
    """Generate all price scenarios for testing"""
    scenarios = []
    
    # Base price
    base_price = 0.1  # Starting AVL price
    
    # Generate scenarios for each day and percentage change
    for day in STRIKE_DAYS:
        for pct_change in PCT_CHANGES:
            # Create unique scenario ID
            scenario_id = f"day{day}_pct{int(pct_change*100)}"
            
            # Generate price series with volatility strike
            price_series = create_price_with_volatility_strike(
                timesteps=SIMULATION_DAYS,
                dt=DELTA_TIME,
                base_price=base_price,
                strike_timestep=day,
                pct_change=pct_change
            )
            
            # Add to scenarios list
            scenarios.append({
                'id': scenario_id,
                'strike_day': day,
                'pct_change': pct_change,
                'price_series': price_series
            })
    
    return scenarios

def run_simulation(scenario):
    """
    Run a simulation with the given price scenario
    This is a simplified version - in practice you would
    run the actual simulation with the codebase
    """
    # Initialize metrics tracking
    metrics = {
        'day': [],
        'avl_price': [],
        'total_tvl': [],
        'avl_tvl': [],
        'eth_tvl': [],
        'btc_tvl': [],
        'avl_budget': [],
        'eth_budget': [],
        'btc_budget': [],
        'panic_withdrawal': []
    }
    
    # In a real implementation, this would run the actual simulation
    # Here we're creating example data based on expected behavior patterns
    
    # Initialize with base simulation state
    initial_tvl = 10e6  # $10M initial TVL
    initial_avl_share = 0.7  # 70% AVL initially
    initial_eth_share = 0.3  # 30% ETH initially
    initial_btc_share = 0.0  # 0% BTC initially (activated later)
    initial_budget = 30e6  # 30M AVL initial budget
    
    # Track previous TVL to detect panic withdrawals
    prev_tvl = initial_tvl
    
    for day in range(SIMULATION_DAYS + 1):
        # Get price for this day
        avl_price = scenario['price_series'][day]
        
        # Calculate TVL based on price elasticity
        # Simplified model: If price increases, TVL increases proportionally but with diminishing returns
        # If price drops, TVL drops more rapidly (representing withdrawals)
        price_ratio = avl_price / scenario['price_series'][0]
        
        # BTC activation occurs on day 180
        btc_active = day >= 180
        
        # TVL changes differently before and after the price shock
        if day < scenario['strike_day']:
            # Normal growth before shock
            tvl_factor = np.sqrt(price_ratio) if price_ratio > 1 else price_ratio
            total_tvl = initial_tvl * (1 + 0.002 * day) * tvl_factor  # Slight natural growth
            
            # Share distribution
            if btc_active:
                avl_share = 0.6
                eth_share = 0.3
                btc_share = 0.1
            else:
                avl_share = 0.7
                eth_share = 0.3
                btc_share = 0.0
                
        else:
            # After shock
            # Calculate impact based on shock magnitude
            days_since_shock = day - scenario['strike_day']
            
            if scenario['pct_change'] < -0.5:  # If price dropped >50%
                # Exponential TVL decay model for large drops (panic withdrawals)
                decay_rate = min(0.1, abs(scenario['pct_change']) * 0.1)  # Withdraw 10% daily at worst
                impact_factor = (1 - decay_rate) ** days_since_shock
                
                # Extreme price drops cause compositional shifts
                if scenario['pct_change'] < -0.7:  # >70% drop
                    avl_share = max(0.2, 0.6 - days_since_shock * 0.01)  # AVL share drops
                    eth_share = min(0.7, 0.3 + days_since_shock * 0.01)  # ETH share rises
                    btc_share = 0.1 if btc_active else 0.0
                else:
                    # Less extreme shifts
                    avl_share = max(0.4, 0.6 - days_since_shock * 0.005)
                    eth_share = min(0.5, 0.3 + days_since_shock * 0.005)
                    btc_share = 0.1 if btc_active else 0.0
                    
            elif scenario['pct_change'] > 0.5:  # If price increased >50%
                # Growth model for price increases (inflows)
                growth_rate = min(0.05, scenario['pct_change'] * 0.05)  # Up to 5% daily growth
                impact_factor = (1 + growth_rate) ** min(days_since_shock, 30)  # Caps after 30 days
                
                # Price increases attract more AVL staking
                avl_share = min(0.8, 0.6 + days_since_shock * 0.005)
                eth_share = max(0.1, 0.3 - days_since_shock * 0.003)
                btc_share = 0.1 if btc_active else 0.0
            else:
                # Moderate changes have less dramatic effects
                impact_factor = 1.0 + scenario['pct_change'] * 0.2
                avl_share = 0.6
                eth_share = 0.3
                btc_share = 0.1 if btc_active else 0.0
            
            # Apply the impact to TVL
            total_tvl = prev_tvl * impact_factor
            
        # Calculate component TVLs
        avl_tvl = total_tvl * avl_share
        eth_tvl = total_tvl * eth_share
        btc_tvl = total_tvl * btc_share
        
        # Rewards budget calculations
        # Budget depletes based on TVL and target yields
        avl_yield = 0.15  # 15% target
        eth_yield = 0.035  # 3.5% target
        btc_yield = 0.02  # 2% target
        
        # Daily reward consumption rate (yearly yield / 365)
        avl_daily_reward = avl_tvl * avl_yield / 365
        eth_daily_reward = eth_tvl * eth_yield / 365
        btc_daily_reward = btc_tvl * btc_yield / 365
        
        # Budget replenishment based on predefined schedule
        # This is a simplified version of the actual replenishment
        replenishment_days = [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360]
        
        avl_replenishment = 0
        eth_replenishment = 0
        btc_replenishment = 0
        
        if day in replenishment_days:
            if day < 180:  # Before BTC activation
                avl_replenishment = 5e6  # Example value
                eth_replenishment = 2e6  # Example value
            else:
                avl_replenishment = 10e6  # Example value
                eth_replenishment = 3e6   # Example value
                btc_replenishment = 3e6   # Example value
        
        # Update budgets (simplified)
        if day == 0:
            # Initialize budgets
            avl_budget = initial_budget * 0.7
            eth_budget = initial_budget * 0.3
            btc_budget = 0
        else:
            # Update budgets with consumption and replenishment
            avl_budget = max(0, metrics['avl_budget'][-1] - avl_daily_reward + avl_replenishment)
            eth_budget = max(0, metrics['eth_budget'][-1] - eth_daily_reward + eth_replenishment)
            btc_budget = max(0, metrics['btc_budget'][-1] - btc_daily_reward + btc_replenishment)
        
        # Detect panic withdrawals (TVL dropped by >5% in a day)
        panic_withdrawal = (prev_tvl - total_tvl) / prev_tvl > 0.05 if prev_tvl > 0 else False
        
        # Store metrics
        metrics['day'].append(day)
        metrics['avl_price'].append(avl_price)
        metrics['total_tvl'].append(total_tvl)
        metrics['avl_tvl'].append(avl_tvl)
        metrics['eth_tvl'].append(eth_tvl)
        metrics['btc_tvl'].append(btc_tvl)
        metrics['avl_budget'].append(avl_budget)
        metrics['eth_budget'].append(eth_budget)
        metrics['btc_budget'].append(btc_budget)
        metrics['panic_withdrawal'].append(panic_withdrawal)
        
        # Update previous TVL for next iteration
        prev_tvl = total_tvl
    
    # Create DataFrame from metrics
    df = pd.DataFrame(metrics)
    
    # Add scenario information
    df['scenario_id'] = scenario['id']
    df['strike_day'] = scenario['strike_day']
    df['pct_change'] = scenario['pct_change']
    
    return df

def analyze_results(all_results):
    """Analyze simulation results and create plots"""
    # Create a Figure with subplots for each strike day
    fig, axes = plt.subplots(3, 2, figsize=(18, 16))
    fig.suptitle("Impact of AVL Price Volatility on System Metrics", fontsize=20)
    
    # Create a colormap for visualizing price changes
    colors = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#ffffbf', 
              '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
    cmap = LinearSegmentedColormap.from_list('custom_cmap', colors, N=len(PCT_CHANGES))
    
    # Define custom normalization for color mapping
    norm = plt.Normalize(-0.9, 2.0)
    
    for i, day in enumerate(STRIKE_DAYS):
        # Filter results for this strike day
        day_results = all_results[all_results['strike_day'] == day]
        
        # Group by percentage change and get final metrics
        grouped = day_results.groupby(['pct_change', 'day']).agg({
            'total_tvl': 'mean',
            'avl_budget': 'mean',
            'panic_withdrawal': 'sum'
        }).reset_index()
        
        # Find days with panic withdrawals
        panic_days = grouped[grouped['panic_withdrawal'] > 0]
        
        # Plot TVL impact
        ax1 = axes[i, 0]
        for pct in PCT_CHANGES:
            subset = grouped[grouped['pct_change'] == pct]
            color = cmap(norm(pct))
            ax1.plot(subset['day'], subset['total_tvl'] / 1e6, color=color, 
                     label=f"{int(pct*100)}%" if i == 0 else "")
            
            # Mark the strike day
            ax1.axvline(x=day, color='black', linestyle='--', alpha=0.3)
            
        ax1.set_title(f"TVL Impact (Strike Day {day})")
        ax1.set_xlabel("Simulation Day")
        ax1.set_ylabel("Total TVL (Millions $)")
        ax1.grid(True, alpha=0.3)
        
        # Plot Budget impact
        ax2 = axes[i, 1]
        for pct in PCT_CHANGES:
            subset = grouped[grouped['pct_change'] == pct]
            color = cmap(norm(pct))
            ax2.plot(subset['day'], subset['avl_budget'] / 1e6, color=color)
            
            # Mark the strike day
            ax2.axvline(x=day, color='black', linestyle='--', alpha=0.3)
        
        ax2.set_title(f"AVL Rewards Budget (Strike Day {day})")
        ax2.set_xlabel("Simulation Day")
        ax2.set_ylabel("AVL Budget (Millions tokens)")
        ax2.grid(True, alpha=0.3)
    
    # Add a color bar for percentage changes
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), orientation='horizontal', pad=0.01, aspect=40)
    cbar.set_label("AVL Price Change %", fontsize=14)
    
    # Add legend to first plot only (to avoid duplication)
    # Format with 5 columns to save space
    handles, labels = axes[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=5, fontsize=10)
    
    # Adjust layout
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    
    # Save the figure
    plt.savefig("avl_price_volatility_analysis.png", dpi=300, bbox_inches='tight')
    plt.close(fig)
    
    # Create a heatmap of panic withdrawal likelihood
    fig2, axes2 = plt.subplots(1, 3, figsize=(20, 6))
    fig2.suptitle("Likelihood of Panic Withdrawals by Price Change Scenario", fontsize=16)
    
    for i, day in enumerate(STRIKE_DAYS):
        # Filter results for this strike day
        day_results = all_results[all_results['strike_day'] == day]
        
        # Create a pivot table of panic withdrawal counts
        pivot = day_results.pivot_table(
            index='pct_change', 
            columns='day',
            values='panic_withdrawal',
            aggfunc='sum'
        )
        
        # Create a binary version where any panic = 1
        pivot_binary = (pivot > 0).astype(int)
        
        # Compute total panic days per scenario
        total_panic_days = pivot_binary.sum(axis=1)
        
        # Plot as a heatmap
        sns.heatmap(
            pivot.iloc[:, day:day+30],  # Look at 30 days after strike
            ax=axes2[i],
            cmap='YlOrRd',
            cbar_kws={'label': 'Panic withdrawal events'}
        )
        axes2[i].set_title(f"Panic Withdrawals (Strike Day {day})")
        axes2[i].set_xlabel("Days after strike")
        axes2[i].set_ylabel("Price change %")
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig("avl_panic_withdrawal_heatmap.png", dpi=300, bbox_inches='tight')
    
    # Create summary metrics table
    summary = pd.DataFrame({
        'strike_day': [],
        'pct_change': [],
        'final_tvl': [],
        'tvl_pct_change': [],
        'final_budget': [],
        'budget_consumed_pct': [],
        'panic_days': []
    })
    
    for day in STRIKE_DAYS:
        for pct in PCT_CHANGES:
            scenario = all_results[(all_results['strike_day'] == day) & 
                                 (all_results['pct_change'] == pct)]
            
            if not scenario.empty:
                initial_tvl = scenario[scenario['day'] == 0]['total_tvl'].values[0]
                final_tvl = scenario[scenario['day'] == SIMULATION_DAYS]['total_tvl'].values[0]
                
                initial_budget = scenario[scenario['day'] == 0]['avl_budget'].values[0]
                final_budget = scenario[scenario['day'] == SIMULATION_DAYS]['avl_budget'].values[0]
                
                panic_count = scenario['panic_withdrawal'].sum()
                
                summary = summary.append({
                    'strike_day': day,
                    'pct_change': pct,
                    'final_tvl': final_tvl,
                    'tvl_pct_change': (final_tvl - initial_tvl) / initial_tvl * 100,
                    'final_budget': final_budget,
                    'budget_consumed_pct': (initial_budget - final_budget) / initial_budget * 100,
                    'panic_days': panic_count
                }, ignore_index=True)
    
    # Save summary to CSV
    summary.to_csv("avl_volatility_summary.csv", index=False)
    
    return summary

def main():
    # Generate scenarios
    print("Generating price scenarios...")
    scenarios = generate_price_scenarios()
    print(f"Created {len(scenarios)} price scenarios")
    
    # Run simulations
    print("Running simulations...")
    results = []
    
    # For demonstration, we'll use sequential processing
    # In practice, you could use multiprocessing
    for scenario in scenarios:
        print(f"Processing scenario: {scenario['id']}")
        scenario_results = run_simulation(scenario)
        results.append(scenario_results)
    
    # Combine all results
    all_results = pd.concat(results)
    
    # Analyze and plot results
    print("Analyzing results and creating plots...")
    summary = analyze_results(all_results)
    
    print("Analysis complete!")
    print("Results saved to:")
    print("- avl_price_volatility_analysis.png")
    print("- avl_panic_withdrawal_heatmap.png")
    print("- avl_volatility_summary.csv")